### 3W dataset feature engineering notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

pd.set_option('max_columns', None)

In [3]:
data_path = Path('data_upgraded_downsampled')
events_names = {0: 'Normal',
                1: 'Abrupt Increase of BSW',
                2: 'Spurious Closure of DHSV',
                3: 'Severe Slugging',
                4: 'Flow Instability',
                5: 'Rapid Productivity Loss',
                6: 'Quick Restriction in PCK',
                7: 'Scaling in PCK',
                8: 'Hydrate in Production Line'
               }
columns = ['P-PDG',
           'P-TPT',
           'T-TPT',
           'P-MON-CKP',
           'T-JUS-CKP',
           'class']
rare_threshold = 0.01

In [4]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [5]:
real_instances = list(class_and_file_generator(data_path, real=True, simulated=False, drawn=False))
simulated_instances = list(class_and_file_generator(data_path, real=False, simulated=True, drawn=False))

In [6]:
#simulirani primjeri za znacajku 'well' imaju vrijednost -1

def load_instance(instance):
    class_code, instance_path = instance
    try:
        well, instance_id = instance_path.stem.split('_')
        if 'WELL' in well:
            well = well.split('-')[1]
        if 'SIMULATED' in well:
            well = '-1'
        df = pd.read_csv(instance_path, index_col='timestamp', parse_dates=['timestamp'])
        assert (df.columns == columns).all(), "invalid columns in the file {}: {}".format(str(instance_path), str(df.columns.tolist()))
        df['class_code'] = class_code
        df['well'] = well
        df['instance_id'] = instance_id
        df = df[['class_code', 'well', 'instance_id'] + columns]
        return df
    except Exception as e:
        raise Exception('error reading file {}: {}'.format(instance_path, e))
        
def load_instances(instances):
    pool = ThreadPool()
    all_df = []
    try:
        for df in pool.imap(load_instance, instances):
            all_df.append(df)
    finally:
        pool.terminate()
    return all_df

In [7]:
#ucitavanje stvarnih i simuliranih csv primjera u df-ove

instances = real_instances + simulated_instances
data = load_instances(instances)
data[0].head(1) #2017-09-25 01:00:30           2017-09-20 11:00:50

,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
timestamp,,,,,,,,,
2018-06-18 06:02:40,1,00006,20180618060245,-1.180116e+42,20787460.0,117.91216,10085830.0,70.869864,0.0


In [8]:
data[1].head(1) #2017-09-18 12:01:00           2017-09-25 01:00:30

,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
timestamp,,,,,,,,,
2017-08-02 12:30:00,1,00006,20170802123000,6733560.0,18144097.0,119.1764,2463897.1,81.072564,0.0


In [9]:
#pretvorba identifikatora dogadaja i oznake busotine u integer tipove podataka

for df in data:
    df['instance_id'] = pd.to_numeric(df['instance_id'])
    df['well'] = pd.to_numeric(df['well'])
    
print(df.dtypes)
data[0].head()

class_code       int64
well             int64
instance_id      int64
P-PDG          float64
P-TPT          float64
T-TPT          float64
P-MON-CKP      float64
T-JUS-CKP      float64
class          float64
dtype: object


,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
timestamp,,,,,,,,,
2018-06-18 06:02:40,1,6,20180618060245,-1.180116e+42,20787460.0,117.91216,10085830.0,70.869864,0.0
2018-06-18 06:02:50,1,6,20180618060245,-1.180116e+42,20787460.0,117.91118,10086711.0,70.869492,0.0
2018-06-18 06:03:00,1,6,20180618060245,-1.180116e+42,20787460.0,117.90989,10087920.0,70.868999,0.0
2018-06-18 06:03:10,1,6,20180618060245,-1.180116e+42,20787460.0,117.90858,10089123.0,70.868505,0.0
2018-06-18 06:03:20,1,6,20180618060245,-1.180116e+42,20787460.0,117.90728,10090330.0,70.868008,0.0


In [10]:
#koliko je null vrijednosti po znacajci u skupu podataka

x = None
for dataFrame in data:
    if x is None:
        x = dataFrame.isna().sum(axis = 0)
    else:
        x += dataFrame.isna().sum(axis = 0)
print(x)

class_code          0
well                0
instance_id         0
P-PDG              62
P-TPT              62
T-TPT          580741
P-MON-CKP      111950
T-JUS-CKP      170107
class               0
dtype: int64


In [11]:
#pretvorba null vrijednosti u konacne vrijednosti uporabom funkcije impute(...)
#dokumentacija https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#tsfresh.utilities.dataframe_functions.impute

import tsfresh as tf
print(data[0].dtypes)
imputed_data = []
for df in data:
    df2 = tf.utilities.dataframe_functions.impute(df.loc[:, df.columns != 'timestamp'])
    imputed_df = df2
    imputed_data.append(imputed_df)

class_code       int64
well             int64
instance_id      int64
P-PDG          float64
P-TPT          float64
T-TPT          float64
P-MON-CKP      float64
T-JUS-CKP      float64
class          float64
dtype: object


/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['P-MON-CKP' 'T-JUS-CKP'] did not have any finite values. Filling with zeros.
  warnings.warn(
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['T-JUS-CKP'] did not have any finite values. Filling with zeros.
  warnings.warn(
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['T-TPT'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [12]:
del data


In [13]:
#provjera jesu li uklonjene sve null vrijednosti

x = None
for df in imputed_data:
    if x is None:
        x = df.isna().sum(axis = 0)
    else:
        x += df.isna().sum(axis = 0)
print(x)

class_code     0
well           0
instance_id    0
P-PDG          0
P-TPT          0
T-TPT          0
P-MON-CKP      0
T-JUS-CKP      0
class          0
dtype: int64


In [14]:
#stvaranje prozora, svaki prozor sadrzi 30 redaka
window_size = 60
data_windowed = []

for df in imputed_data:
    for i in range(df.shape[0]//window_size):
        window = df.iloc[i*window_size:(i+1)*window_size]
        data_windowed.append(window)

In [15]:
all_windowed = pd.concat(data_windowed, keys=range(0, len(data_windowed))).reset_index()

In [16]:
window_classes = []
for frame in data_windowed:
    window_classes.append(frame['class_code'][0])

In [17]:
print(len(window_classes))
print(len(data_windowed))
df_classes = df = pd.DataFrame(window_classes)
df_classes.rename(columns = {0 : 'class_code'}, inplace = True)
df_classes

78745
78745


,class_code
0,1
1,1
2,1
3,1
4,1
...,...
78740,3
78741,3
78742,3
78743,3


In [18]:
all_windowed.rename(columns = {'level_0':'window_id'}, inplace=True) #maknut timestamp, ne čini se više potrebnim
all_windowed.isna().sum(axis=0)

window_id      0
timestamp      0
class_code     0
well           0
instance_id    0
P-PDG          0
P-TPT          0
T-TPT          0
P-MON-CKP      0
T-JUS-CKP      0
class          0
dtype: int64

In [19]:
print(all_windowed.shape)
all_windowed.dtypes

(4724700, 11)


window_id               int64
timestamp      datetime64[ns]
class_code              int64
well                    int64
instance_id             int64
P-PDG                 float64
P-TPT                 float64
T-TPT                 float64
P-MON-CKP             float64
T-JUS-CKP             float64
class                 float64
dtype: object

In [20]:
params = {
    'mean': None,
    'variance': None,
    'skewness': None,
    'kurtosis': None,
    'fft_aggregated': [{'aggtype': 'centroid'},
  {'aggtype': 'variance'},
  {'aggtype': 'skew'},
  {'aggtype': 'kurtosis'}], 
    'maximum': None,
    'minimum': None,
    'median': None,
    'quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],
    'variation_coefficient': None,
    'mean_change': None,
    'mean_second_derivative_central': None,
    'friedrich_coefficients': [
  {'coeff': 1, 'm': 3, 'r': 30},
  {'coeff': 3, 'm': 3, 'r': 30}],
 'longest_strike_above_mean': None,
 'longest_strike_below_mean': None,
 'fourier_entropy': [{'bins': 2}],
 'large_standard_deviation': [{'r': 0.1},
  {'r': 0.5},
  {'r': 0.9}],
'fft_coefficient': [{'attr': 'real', 'coeff': 0},
  {'attr': 'real', 'coeff': 1},
  {'attr': 'real', 'coeff': 2},
  {'attr': 'imag', 'coeff': 0},
  {'attr': 'imag', 'coeff': 1},
  {'attr': 'imag', 'coeff': 2},
  {'attr': 'abs', 'coeff': 0},
  {'attr': 'abs', 'coeff': 1},
  {'attr': 'abs', 'coeff': 2},
  {'attr': 'angle', 'coeff': 0},
  {'attr': 'angle', 'coeff': 1},
  {'attr': 'angle', 'coeff': 2}]}

In [21]:
all_windowed.head(1)

,window_id,timestamp,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
0,0,2018-06-18 06:02:40,1,6,20180618060245,-1.180116e+42,20787460.0,117.91216,10085830.0,70.869864,0.0


In [22]:
all_windowed.drop(['class_code', 'well', 'class', 'instance_id'], axis='columns', inplace=True)
all_windowed.head(1)

,window_id,timestamp,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP
0,0,2018-06-18 06:02:40,-1.180116e+42,20787460.0,117.91216,10085830.0,70.869864


In [23]:
number_of_windows = 90000//window_size
df_classes
for i in range(0, all_windowed.shape[0]//90000):
    path=Path('data_upgraded_features/60/'+str(i)+'.csv')
    test_df = all_windowed[i*90000:(i+1)*90000]
    feats_df = tf.extract_features(test_df, default_fc_parameters=params, column_id="window_id", column_sort="timestamp", column_kind=None, column_value=None)
    class_df = df_classes[i*number_of_windows:(i+1)*number_of_windows]
    result = pd.concat([feats_df, class_df], axis=1, ignore_index=False)
    result.to_csv(path)
path=Path('data_upgraded_features/60/'+str(i+1)+'.csv')
test_df = all_windowed[(i+1)*90000:]
feats_df = tf.extract_features(test_df, default_fc_parameters=params, column_id="window_id", column_sort="timestamp", column_kind=None, column_value=None)
class_df = df_classes[(i+1)*number_of_windows:]
result = pd.concat([feats_df, class_df], axis=1, ignore_index=False)
result.to_csv(path)

Feature Extraction: 100%|██████████| 10/10 [00:20<00:00,  2.06s/it]


In [24]:
result

,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-PDG__friedrich_coefficients__coeff_3__m_3__r_30,P-PDG__longest_strike_above_mean,P-PDG__longest_strike_below_mean,P-PDG__fourier_entropy__bins_2,P-PDG__large_standard_deviation__r_0.1,P-PDG__large_standard_deviation__r_0.5,P-PDG__large_standard_deviation__r_0.9,"P-PDG__fft_coefficient__attr_""real""__coeff_0","P-PDG__fft_coefficient__attr_""real""__coeff_1","P-PDG__fft_coefficient__attr_""real""__coeff_2","P-PDG__fft_coefficient__attr_""imag""__coeff_0","P-PDG__fft_coefficient__attr_""imag""__coeff_1","P-PDG__fft_coefficient__attr_""imag""__coeff_2","P-PDG__fft_coefficient__attr_""abs""__coeff_0","P-PDG__fft_coefficient__attr_""abs""__coeff_1","P-PDG__fft_coefficient__attr_""abs""__coeff_2","P-PDG__fft_coefficient__attr_""angle""__coeff_0","P-PDG__fft_coefficient__attr_""angle""__coeff_1","P-PDG__fft_coefficient__attr_""angle""__coeff_2",P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,P-TPT__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__friedrich_coefficients__coeff_3__m_3__r_30,P-TPT__longest_strike_above_mean,P-TPT__longest_strike_below_mean,P-TPT__fourier_entropy__bins_2,P-TPT__large_standard_deviation__r_0.1,P-TPT__large_standard_deviation__r_0.5,P-TPT__large_standard_deviation__r_0.9,"P-TPT__fft_coefficient__attr_""real""__coeff_0","P-TPT__fft_coefficient__attr_""real""__coeff_1","P-TPT__fft_coefficient__attr_""real""__coeff_2","P-TPT__fft_coefficient__attr_""imag""__coeff_0","P-TPT__fft_coefficient__attr_""imag""__coeff_1","P-TPT__fft_coefficient__attr_""imag""__coeff_2","P-TPT__fft_coefficient__attr_""abs""__coeff_0","P-TPT__fft_coefficient__attr_""abs""__coeff_1","P-TPT__fft_coefficient__attr_""abs""__coeff_2","P-TPT__fft_coefficient__attr_""angle""__coeff_0","P-TPT__fft_coefficient__attr_""angle""__coeff_1","P-TPT__fft_coefficient__attr_""angle""__coeff_2",T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,"T-TPT__fft_aggregated__aggtype_""centroid""","T-TPT__fft_aggregated__aggtype_""variance""","T-TPT__fft_aggregated__aggtype_""skew""","T-TPT__fft_aggregated__aggtype_""kurtosis""",T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,T-TPT__friedrich_coefficients__coeff_1__m_3__r_30,T-TPT__friedrich_coefficients__coeff_3__m_3__r_30,T-TPT__longest_strike_above_mean,T-TPT__longest_strike_below_mean,T-TPT__fourier_entropy__bins_2,T-TPT__large_standard_deviation__r_0.1,T-TPT__large_standard_deviation__r_0.5,T-TPT__large_standard_deviation__r_0.9,"T-TPT__fft_coefficient__attr_""real""__coeff_0","T-TPT__fft_coefficient__attr_""real""__coeff_1","T-TPT__fft_coefficient__attr_""real""__coeff_2","T-TPT__fft_coefficient__attr_""imag""__coeff_0","T-TPT__fft_coefficient